In [1]:
# import nbimporter
# import climate

# 日よけ効果係数算定ツール Test


- 一次目標：日よけ効果係数算定ツールのpython上での再現


## A. 窓まわりの設定(仕様書5)と太陽位置の算定(仕様書6.2)

### A.1 日時

- 日付`NDay`は1/1からの通しの日数
- 時刻`NHour`は整数, 0時～23時が基本。12/31のみ24時あり
- 時刻`TT`は、時間分割`MM`における時刻
  - `MM`：1時間の内の$(1/$ `NDT`$)$間隔の順番, 正時が`MM`$=0$, `MM`$=0～$ `NDT`$-1$


- 時刻`Hour01`は、前時刻の`MM` $=$ `NDT`$/2～$同時刻の`MM` $=$ `NDT`$/2-1$の時間分割を、その正時に属するものとして扱うための時刻 → 時間毎の効果係数を算定するために使用するだけ

In [2]:
""" 日時 Test """
# \確認.xlsx "NDayNHour"シート → \NDayNHour.csv を作成 → \TestConfig01 下に置いて読み込み → 計算

import Shading_Correction_Factor_Modules as SCFModule
import pandas as pd
import sys
csv_input = pd.read_csv(filepath_or_buffer="./TestConfig01/NDayNHour.csv", encoding="ms932", sep=",")
if csv_input.columns[0]!="Hour00":
    sys.exit("ファイル内に貼り付けたテスト条件が違います")
for i in range(len(csv_input)):
    [Hour00, NDayA, NHourA] = csv_input.values[i]
    [NDay, NHour] = SCFModule.calc_NDayNHour(Hour00)
    print('Hour00 = {}: NDay = {}, 期待値 = {}, 残差 = {}, NHour = {}, 期待値 = {}, 残差 = {}'
          .format( Hour00, NDay, NDayA, NDay - NDayA, NHour, NHourA, NHour - NHourA ))

ModuleNotFoundError: No module named 'Shading_Correction_Factor_Modules'

In [ ]:
""" 時刻TT Test """

import Shading_Correction_Factor_Modules as SCFModule
NDT=6
for NHour in range(0,24):
    for MM in range(0,NDT):
        print(NHour, MM, SCFModule.calc_TT(NHour, NDT, MM))

### A.2 赤緯の計算 (仕様書6.2 式(4))

- 赤緯$\delta_d [deg]$, $N$: 1月1日を$N=1$とした年頭からの通しの日数$[day]$
  - 右辺の余弦のかっこ内の角度は$radian$単位となっているので注意
  
$$ \begin{align}
\delta_d = (180 / \pi) & \{0.006322 - 0.405748 \cos (2 \pi N / 366 + 0.153231)\\
& - 0.005880 \cos (4 \pi N / 366 + 0.207099)\\
& - 0.003233 \cos (6 \pi N / 366 + 0.620129) \} \qquad \qquad \qquad (4) \\
\end{align} $$

In [ ]:
""" 式(4) Test """
# \確認.xlsx "式(4)deltad"シート → \deltad.csv を作成 → \TestConfig01 下に置いて読み込み → 計算

import Shading_Correction_Factor_Modules as SCFModule
import pandas as pd
import sys
csv_input = pd.read_csv(filepath_or_buffer="./TestConfig01/deltad.csv", encoding="ms932", sep=",")
if csv_input.columns[0]!="deltad_case":
    sys.exit("ファイル内に貼り付けたテスト条件が違います")
for i in range(len(csv_input)):
    [case, NDay, deltadA] = csv_input.values[i]
    deltad = SCFModule.calc_deltad(NDay)
    print('case{}: deltad = {}, 期待値 = {}, 残差 = {}'.format( int(case), deltad, deltadA, deltad - deltadA ))    

### A.3 均時差の計算 (仕様書6.2 式(6))

- 均時差$e_d[hour]$, $N$: 1月1日を$N=1$とした年頭からの通しの日数$[day]$
  - 右辺の余弦のかっこ内の角度は$radian$単位となっているので注意

$$ \begin{align}
e_d = -0.000279 &+ 0.122772 \cos (2 \pi N / 366 + 1.498311)\\
& - 0.165458 \cos (4 \pi N / 366 - 1.261546)\\
& - 0.005354 \cos (6 \pi N / 366 - 1.1571) \} \qquad \qquad \qquad (6) \\
\end{align} $$

In [ ]:
""" 式(6) Test """
# \確認.xlsx "式(6)eed"シート → \eed.csv を作成 → \TestConfig01 下に置いて読み込み → 計算

import Shading_Correction_Factor_Modules as SCFModule
import pandas as pd
import sys
csv_input = pd.read_csv(filepath_or_buffer="./TestConfig01/eed.csv", encoding="ms932", sep=",")
if csv_input.columns[0]!="eed_case":
    sys.exit("ファイル内に貼り付けたテスト条件が違います")
for i in range(len(csv_input)):
    [case, NDay, eedA] = csv_input.values[i]
    eed = SCFModule.calc_eed(NDay)
    print('case{}: eed = {}, 期待値 = {}, 残差 = {}'.format( int(case), eed, eedA, eed - eedA ))   

### A.4 時角の計算 (仕様書6.2 式(7))

- 時角$T_{d,t}[deg]$, 時刻$t[hour]$, 均時差$e_d[hour]$, 経度$L[deg]$

$$T_{d,t} = (t + e_d - 12) \times 15 + (L - 135)\qquad (7) $$

In [ ]:
""" 式(7) Test """
# \確認.xlsx "式(7)Tdt"シート → \Tdt.csv を作成 → \TestConfig01 下に置いて読み込み → 計算

import Shading_Correction_Factor_Modules as SCFModule
import pandas as pd
import sys
csv_input = pd.read_csv(filepath_or_buffer="./TestConfig01/Tdt.csv", encoding="ms932", sep=",")
if csv_input.columns[0]!="Tdt_case":
    sys.exit("ファイル内に貼り付けたテスト条件が違います")
for i in range(len(csv_input)):
    [case, Longitude, NDay, TT, TdtA] = csv_input.values[i]
    Tdt = SCFModule.calc_Tdt(Longitude, SCFModule.calc_eed(NDay), TT)
    print('case{}: Tdt = {}, 期待値 = {}, 残差 = {}'.format( int(case), Tdt, TdtA, Tdt - TdtA ))   

### A.5 太陽高度の正弦の計算 (仕様書6.2 式(8))

- 太陽高度$h_{S,d,t}[deg]$, 緯度$\phi[deg]$, 赤緯$\delta_d[deg]$,時角$T_{d,t}[deg]$

$$\sin h_{S,d,t} = max[0, \sin \phi \sin \delta_d + \cos \phi \cos \delta_d \cos T_{d,t}] \qquad (8) $$

In [ ]:
""" 式(8) Test """
# \確認.xlsx "式(8)sinh,式(9)cosh"シート → \sinh.csv を作成 → \TestConfig01 下に置いて読み込み → 計算

import Shading_Correction_Factor_Modules as SCFModule
import pandas as pd
import sys
csv_input = pd.read_csv(filepath_or_buffer="./TestConfig01/sinh.csv", encoding="ms932", sep=",")
if csv_input.columns[0]!="sinh_case":
    sys.exit("ファイル内に貼り付けたテスト条件が違います")
for i in range(len(csv_input)):
    [case, Latitude, Longitude, NDay, TT, sinhA] = csv_input.values[i]
    sinh = SCFModule.calc_sinh(Latitude, SCFModule.calc_deltad(NDay)
                               , SCFModule.calc_Tdt(Longitude, SCFModule.calc_eed(NDay), TT) )
    print('case{}: sinh = {}, 期待値 = {}, 残差 = {}'.format( int(case), sinh, sinhA, sinh - sinhA ))   
    

### A.6 太陽高度とその余弦の計算 (仕様書6.2 式(9))

- 太陽高度$h_{S,d,t}[deg]$

$$\cos h_{S,d,t} = (1 - \sin ^2 h_{S,d,t})^{0.5} \qquad (9) $$

- 式(8), (9)より、$h_{S,d,t} = \tan^{-1} (\sin h_{S,d,t} / \cos h_{S,d,t})$

In [ ]:
""" 式sdt(calc_co(9)+α Test """
# \確認.xlsx "式(8)～式(12)"シート → \cosh.csv を作成 → \TestConfig01 下に置いて読み込み → 計算

import Shading_Correction_Factor_Modules as SCFModule
import pandas as pd
import sys
csv_input = pd.read_csv(filepath_or_buffer="./TestConfig01/cosh.csv", encoding="ms932", sep=",")
if csv_input.columns[0]!="cosh_case":
    sys.exit("ファイル内に貼り付けたテスト条件が違います")
for i in range(len(csv_input)):
    [case, Latitude, Longitude, NDay, TT, coshA] = csv_input.values[i]
    sinh = SCFModule.calc_sinh(Latitude, SCFModule.calc_deltad(NDay)
                               , SCFModule.calc_Tdt(Longitude, SCFModule.calc_eed(NDay), TT) )
    cosh = SCFModule.calc_cosh(sinh)
    print('case{}: cosh = {}, 期待値 = {}, 残差 = {}'.format( int(case), cosh, coshA, cosh - coshA ))   

print()    

csv_input = pd.read_csv(filepath_or_buffer="./TestConfig01/hsdt.csv", encoding="ms932", sep=",")
if csv_input.columns[0]!="hsdt_case":
    sys.exit("ファイル内に貼り付けたテスト条件が違います")
for i in range(len(csv_input)):
    [case, Latitude, Longitude, NDay, TT, hsdtA] = csv_input.values[i]
    sinh = SCFModule.calc_sinh(Latitude, SCFModule.calc_deltad(NDay)
                               , SCFModule.calc_Tdt(Longitude, SCFModule.calc_eed(NDay), TT) )
    cosh = SCFModule.calc_cosh(sinh)
    hsdt = SCFModule.calc_hsdt(cosh, sinh)
    print('case{}: hsdt = {}, 期待値 = {}, 残差 = {}'.format( int(case), hsdt, hsdtA, hsdt - hsdtA ))   

### A.7 太陽方位角の計算 (仕様書6.2 式(10)～(12))

- 太陽方位角$A_{ZS,d,t}[deg]$, 太陽高度$h_{S,d,t}[deg]$, 赤緯$\delta_d[deg]$, 時角$T_{d,t}[deg]$, 緯度$\phi[deg]$

$$\sin A_{ZS,d,t} = \cos \delta_d \sin T_{d,t} / \cos h_{S,d,t} \qquad (10) $$

$$\cos A_{ZS,d,t} = (\sin h_{S,d,t} \sin \phi - \sin \delta_d) / (\cos h_{S,d,t} \cos \phi) \qquad (11) $$

$$
A_{ZS,d,t} = \left\{
\begin{array}{ll}
\tan^{-1} (\sin A_{ZS,d,t} / \cos A_{ZS,d,t}) + 180 \hspace{24pt} (\sin A_{ZS,d,t} > 0, \cos A_{ZS,d,t} < 0)
\\
\tan^{-1} (\sin A_{ZS,d,t} / \cos A_{ZS,d,t}) - 180 \hspace{24pt} (\sin A_{ZS,d,t} < 0, \cos A_{ZS,d,t} < 0)
\\
90 \hspace{136pt} (\sin A_{ZS,d,t} = 1, \cos A_{ZS,d,t} = 0)
\\
-90 \hspace{130pt} (\sin A_{ZS,d,t} = -1, \cos A_{ZS,d,t} = 0)
\\
\tan^{-1} (\sin A_{ZS,d,t} / \cos A_{ZS,d,t}) \hspace{48pt} (other)
\end{array}
\right.  \qquad (12) 
$$


In [ ]:
""" 式(10)～(12) Test """
# \確認.xlsx "式(8)～式(12)"シート → \Azsdt.csv を作成 → \TestConfig01 下に置いて読み込み → 計算

import Shading_Correction_Factor_Modules as SCFModule
import pandas as pd
import sys
csv_input = pd.read_csv(filepath_or_buffer="./TestConfig01/Azsdt.csv", encoding="ms932", sep=",")
if csv_input.columns[0]!="Azsdt_case":
    sys.exit("ファイル内に貼り付けたテスト条件が違います")
for i in range(len(csv_input)):
    [case, Latitude, Longitude, NDay, TT, AzsdtA] = csv_input.values[i]
    deltad = SCFModule.calc_deltad(NDay)
    eed = SCFModule.calc_eed(NDay)
    Tdt = SCFModule.calc_Tdt(Longitude, eed, TT) 
    sinh = SCFModule.calc_sinh(Latitude, deltad, Tdt)
    cosh = SCFModule.calc_cosh(sinh)
    Azsdt = SCFModule.calc_Azsdt(Latitude, deltad, Tdt, sinh, cosh)
    print('case{}: Azsdt = {}, 期待値 = {}, 残差 = {}'.format( int(case), Azsdt, AzsdtA, Azsdt - AzsdtA ))  

### A.8 窓面の方位 (仕様書5.2 図4)

- 窓面の方位は、以下の通り
  - 北北東：$-157.5°$, 北東：$-135°$, …, 東：$-90°$, …, 南：$0°$, …, 西：$+90°$, …,北：$+180°$
  - 角度指定も可：$-180°< A_{ZW,j} \leq +180°$
  - デフォルトは8方位指定

In [6]:
""" 窓面の方位 (仕様書5. 図4) Test """
# \確認.xlsx "窓面方位Azwj"シート → \Azwj.csv を作成 → \TestConfig01 下に置いて読み込み → 計算

import Shading_Correction_Factor_Modules as SCFModule
import pandas as pd
import sys
csv_input = pd.read_csv(filepath_or_buffer="./TestConfig01/Azwj.csv", encoding="ms932", sep=",")
if csv_input.columns[0]!="方位":
    sys.exit("ファイル内に貼り付けたテスト条件が違います")
for i in range(len(csv_input)):
    [Azimuth, AzwjA] = csv_input.values[i]
    Azwj = SCFModule.calc_Azwj(Azimuth)
    print('方位：{}: Azwj = {}, 期待値 = {}, 残差 = {}'.format( Azimuth, Azwj, AzwjA, Azwj - AzwjA ))  

方位：北北東: Azwj = -157.5, 期待値 = -157.5, 残差 = 0.0
方位：北東: Azwj = -135.0, 期待値 = -135.0, 残差 = 0.0
方位：東北東: Azwj = -112.5, 期待値 = -112.5, 残差 = 0.0
方位：東: Azwj = -90.0, 期待値 = -90.0, 残差 = 0.0
方位：東南東: Azwj = -67.5, 期待値 = -67.5, 残差 = 0.0
方位：南東: Azwj = -45.0, 期待値 = -45.0, 残差 = 0.0
方位：南南東: Azwj = -22.5, 期待値 = -22.5, 残差 = 0.0
方位：南: Azwj = 0.0, 期待値 = 0.0, 残差 = 0.0
方位：南南西: Azwj = 22.5, 期待値 = 22.5, 残差 = 0.0
方位：南西: Azwj = 45.0, 期待値 = 45.0, 残差 = 0.0
方位：西南西: Azwj = 67.5, 期待値 = 67.5, 残差 = 0.0
方位：西: Azwj = 90.0, 期待値 = 90.0, 残差 = 0.0
方位：西北西: Azwj = 112.5, 期待値 = 112.5, 残差 = 0.0
方位：北西: Azwj = 135.0, 期待値 = 135.0, 残差 = 0.0
方位：北北西: Azwj = 157.5, 期待値 = 157.5, 残差 = 0.0
方位：北: Azwj = 180.0, 期待値 = 180.0, 残差 = 0.0
方位：-179: Azwj = -179.0, 期待値 = -179.0, 残差 = 0.0
方位：180: Azwj = 180.0, 期待値 = 180.0, 残差 = 0.0


SystemExit: 窓面方位の入力が不適切です

C:\Users\miura\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:
""" 窓面の方位 (仕様書5. 図4) Test """
# \確認.xlsx "窓面方位Azwj"シート → \Azwj.csv を作成 → \TestConfig01 下に置いて読み込み → 計算

import Shading_Correction_Factor_Modules as SCFModule
import pandas as pd
import sys
csv_input = pd.read_csv(filepath_or_buffer="./TestConfig01/Azwj.csv", encoding="ms932", sep=",")
if csv_input.columns[0]!="方位":
    sys.exit("ファイル内に貼り付けたテスト条件が違います")
for i in range(len(csv_input)):
    [Azimuth, AzwjA] = csv_input.values[i]
    Azwj = SCFModule.calc_Azwj(Azimuth)
    print('方位：{}: Azwj = {}, 期待値 = {}, 残差 = {}'.format( Azimuth, Azwj, AzwjA, Azwj - AzwjA ))  

方位：北北東: Azwj = -157.5, 期待値 = -157.5, 残差 = 0.0
方位：北東: Azwj = -135.0, 期待値 = -135.0, 残差 = 0.0
方位：東北東: Azwj = -112.5, 期待値 = -112.5, 残差 = 0.0
方位：東: Azwj = -90.0, 期待値 = -90.0, 残差 = 0.0
方位：東南東: Azwj = -67.5, 期待値 = -67.5, 残差 = 0.0
方位：南東: Azwj = -45.0, 期待値 = -45.0, 残差 = 0.0
方位：南南東: Azwj = -22.5, 期待値 = -22.5, 残差 = 0.0
方位：南: Azwj = 0.0, 期待値 = 0.0, 残差 = 0.0
方位：南南西: Azwj = 22.5, 期待値 = 22.5, 残差 = 0.0
方位：南西: Azwj = 45.0, 期待値 = 45.0, 残差 = 0.0
方位：西南西: Azwj = 67.5, 期待値 = 67.5, 残差 = 0.0
方位：西: Azwj = 90.0, 期待値 = 90.0, 残差 = 0.0
方位：西北西: Azwj = 112.5, 期待値 = 112.5, 残差 = 0.0
方位：北西: Azwj = 135.0, 期待値 = 135.0, 残差 = 0.0
方位：北北西: Azwj = 157.5, 期待値 = 157.5, 残差 = 0.0
方位：北: Azwj = 180.0, 期待値 = 180.0, 残差 = 0.0
方位：-179: Azwj = -179.0, 期待値 = -179.0, 残差 = 0.0
方位：180: Azwj = 180.0, 期待値 = 180.0, 残差 = 0.0


SystemExit: 窓面方位の入力が不適切です

C:\Users\miura\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### A.9 窓面の法線ベクトルと太陽位置とのなす水平面上の角度の計算 (仕様書6.2 式(1))

- 窓面の法線ベクトルと太陽位置とのなす水平面上の角度$A_{ZW,j,d,t}[deg]$, 太陽方位角$A_{ZS,d,t}[deg]$, 外壁$j$の方位角$A_{ZW,j}[deg]$

$$
A_{ZW,j,d,t} = \left\{
\begin{array}{ll}
A_{ZS,d,t} - A_{ZW,j} \hspace{48pt} (-180 < A_{ZS,d,t} - A_{ZW,j} \leq 180)
\\
A_{ZS,d,t} - A_{ZW,j} + 360 \hspace{24pt} (A_{ZS,d,t} - A_{ZW,j} \leq -180)
\\
A_{ZS,d,t} - A_{ZW,j} - 360 \hspace{24pt} (A_{ZS,d,t} - A_{ZW,j} \geq 180)
\end{array}
\right.  \qquad (1) 
$$

In [ ]:
""" 式(1) Test """
# \確認.xlsx "式(1)Azwjdt"シート → \Azwjdt.csv を作成 → \TestConfig01 下に置いて読み込み → 計算

import Shading_Correction_Factor_Modules as SCFModule
import pandas as pd
import sys
csv_input = pd.read_csv(filepath_or_buffer="./TestConfig01/Azwjdt.csv", encoding="ms932", sep=",")
if csv_input.columns[0]!="Azwjdt_case":
    sys.exit("ファイル内に貼り付けたテスト条件が違います")
for i in range(len(csv_input)):
    [case, Azwj, Latitude, Longitude, NDay, TT, AzwjdtA] = csv_input.values[i]
    deltad = SCFModule.calc_deltad(NDay)
    eed = SCFModule.calc_eed(NDay)
    Tdt = SCFModule.calc_Tdt(Longitude, eed, TT) 
    sinh = SCFModule.calc_sinh(Latitude, deltad, Tdt)
    cosh = SCFModule.calc_cosh(sinh)
    Azsdt = SCFModule.calc_Azsdt(Latitude, deltad, Tdt, sinh, cosh)
    Azwjdt = SCFModule.calc_Azwjdt(Azwj, Azsdt)
    print('case{}: Azwjdt = {}, 期待値 = {}, 残差 = {}'.format( int(case), Azwjdt, AzwjdtA, Azwjdt - AzwjdtA ))  

### A.10 窓まわり寸法のデータの持たせ方デフォルト (仕様書5.1 図2)

In [ ]:
""" set_WSSize Test """
# \確認.xlsx "式(1)WSSize"シート → \WSSize.csv を作成 → \TestConfig01 下に置いて読み込み → 計算

import Shading_Correction_Factor_Modules as SCFModule
import pandas as pd
import sys
csv_input = pd.read_csv(filepath_or_buffer="./TestConfig01/WSSize.csv", encoding="ms932", sep=",")
if csv_input.columns[0]!="WSSize_case":
    sys.exit("ファイル内に貼り付けたテスト条件が違います")
for i in range(len(csv_input)):
    case = csv_input.values[i][0]
    WSSizeA = csv_input.values[i][1:19]
    # print(WSSizeA)
    WSSize1 = csv_input.values[i][19:]
    WSSize = SCFModule.set_WSSize(WSSize1)
    # print(WSSize)
    difWSSize = WSSize - WSSizeA
    max(max(difWSSize),abs(min(difWSSize)))
    
    print('case{}: 残差 = {}'.format( int(case), max(max(difWSSize),abs(min(difWSSize))) ))  

## B. 直達日射が窓に射す面積の計算 (仕様書6.3)

### B.1 太陽がx+側に位置する際のオーバーハングによる影の面積の算定式 (仕様書6.3.1 式(15))

$$
A_{oh0+}(x,y) = \left\{
\begin{array}{ll}
0 \hspace{24pt}(z_{y+}=0)
\\
\dfrac{1}{2} (x_{3y+} + x_2 / 2 - x) \dfrac{z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}}{z_{y+} \tan | A_{ZW,j,d,t} |} (x_{3y+} + x_2 / 2 - x)
\\
\hspace{30pt} \left( \begin{array}{ll}
x_{3y+} + x_2 / 2 - x < z_{y+} \tan | A_{ZW,j,d,t} | \\
y_{1} + y_2 / 2 - y \geq \dfrac{z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}}{z_{y+} \tan | A_{ZW,j,d,t} |} (x_{3y+} + x_2 / 2 - x) 
\end{array} \right)
\\
\Bigl\{ (x_{3y+} + x_2 / 2 - x) - \dfrac{1}{2} (y_{1} + y_2 / 2 - y) \frac{z_{y+} \tan | A_{ZW,j,d,t} |}{z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}} \Bigr\} (y_{1} + y_2 / 2 - y)
\\
\hspace{30pt} \left( \begin{array}{ll}
x_{3y+} + x_2 / 2 - x > \dfrac{z_{y+} \tan | A_{ZW,j,d,t} |}{z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}} (y_{1} + y_2 / 2 - y) \\
y_{1} + y_2 / 2 - y < z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}
\end{array} \right)
\\
( x_{3y+} + x_2 / 2 - x - \dfrac{1}{2} z_{y+} \tan | A_{ZW,j,d,t} | ) z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}
\\
\hspace{30pt} \left( \begin{array}{ll}
x_{3y+} + x_2 / 2 - x \geq z_{y+} \tan | A_{ZW,j,d,t} | \\
y_{1} + y_2 / 2 - y \geq z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}
\end{array} \right)
\end{array}
\right.  \qquad (15) 
$$

In [ ]:
""" 式(15) Test """
# \確認.xlsx "式(15)Aoh0p"シート → \Aoh0p.csv を作成 → \TestConfig01 下に置いて読み込み → 計算

import Shading_Correction_Factor_Modules as SCFModule
import pandas as pd
import sys
csv_input = pd.read_csv(filepath_or_buffer="./TestConfig01/Aoh0p.csv", encoding="ms932", sep=",")
if csv_input.columns[0]!="Aoh0p_case":
    sys.exit("ファイル内に貼り付けたテスト条件が違います")
for i in range(len(csv_input)):
    [case, XX, YY]= csv_input.values[i][0:3]
    WSSize = csv_input.values[i][3:-3]
    [Azw, hs, Aoh0pA] = csv_input.values[i][-3:]
    Aoh0p = SCFModule.calc_Aoh0p(XX, YY, WSSize, Azw, hs)

    print('case{}: Aohop = {}, 期待値 = {}, 残差 = {}'.format( int(case), Aoh0p, Aoh0pA, Aoh0p - Aoh0pA ))    

### B.2 太陽がx+側に位置する際のサイドフィンによる影の面積の算定式 (仕様書6.3.1 式(16))

$$
A_{sf0+}(x,y) = \left\{
\begin{array}{ll}
0 \hspace{24pt}(z_{x+}=0)
\\
\dfrac{1}{2} (y_{1x+} + y_2 / 2 - y) \dfrac{z_{x+} \tan | A_{ZW,j,d,t} |}{z_{x+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}} (y_{1x+} + y_2 / 2 - y)
\\
\hspace{30pt} \left( \begin{array}{ll}
y_{1x+} + y_2 / 2 - y < z_{x+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t} \\
x_{3} + x_2 / 2 - x \geq \dfrac{z_{x+} \tan | A_{ZW,j,d,t} |}{z_{x+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}} (y_{1x+} + y_2 / 2 - y) 
\end{array} \right)
\\
\Bigl\{ (y_{1x+} + y_2 / 2 - y) - \dfrac{1}{2} (x_{3} + x_2 / 2 - x) \dfrac{z_{x+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}}{z_{x+} \tan | A_{ZW,j,d,t} |} \Bigr\} (x_{3} + x_2 / 2 - x)
\\
\hspace{30pt} \left( \begin{array}{ll}
y_{1x+} + y_2 / 2 - y > \dfrac{z_{x+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}}{z_{x+} \tan | A_{ZW,j,d,t} |} (x_{3} + x_2 / 2 - x) \\
x_{3} + x_2 / 2 - x < z_{x+} \tan | A_{ZW,j,d,t} |
\end{array} \right)
\\
( y_{1x+} + y_2 / 2 - y - \dfrac{1}{2} z_{x+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t} ) \; z_{x+} \tan | A_{ZW,j,d,t} |
\\
\hspace{30pt} \left( \begin{array}{ll}
y_{1x+} + y_2 / 2 - y \geq z_{x+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t} \\
x_{3} + x_2 / 2 - x \geq z_{x+} \tan | A_{ZW,j,d,t} |
\end{array} \right)
\end{array}
\right.  \qquad (16) 
$$

- コード中では、座標を入れ替えて、`calc_Aoh0p00`を叩くことで対応
  - 式$(15)$の変数 → 式$(16)$の変数
  - $x$ → $y$
  - $x_2$ → $y_2$
  - $x_{3y+}$ → $y_{1x+}$
  - $y$ → $x$
  - $y_1$ → $x_3$
  - $y_2$ → $x_2$
  - $z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}$ → $z_{x+} \tan | A_{ZW,j,d,t} |$
  - $z_{y+} \tan | A_{ZW,j,d,t} |$ → $z_{x+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}$

In [ ]:
""" 式(16) Test """
# \確認.xlsx "式(16)Asf0p"シート → \Asf0p.csv を作成 → \TestConfig01 下に置いて読み込み → 計算

import Shading_Correction_Factor_Modules as SCFModule
import pandas as pd
import sys
csv_input = pd.read_csv(filepath_or_buffer="./TestConfig01/Asf0p.csv", encoding="ms932", sep=",")
if csv_input.columns[0]!="Asf0p_case":
    sys.exit("ファイル内に貼り付けたテスト条件が違います")
for i in range(len(csv_input)):
    [case, XX, YY]= csv_input.values[i][0:3]
    WSSize = csv_input.values[i][3:-3]
    [Azw, hs, Asf0pA] = csv_input.values[i][-3:]
    Asf0p = SCFModule.calc_Asf0p(XX, YY, WSSize, Azw, hs)

    print('case{}: Asfop = {}, 期待値 = {}, 残差 = {}'.format( int(case), Asf0p, Asf0pA, Asf0p - Asf0pA ))   

### B.3 太陽がx+側に位置する際の日射が射す部分の面積の計算式 (仕様書6.3.1 式(14))

$$ \begin{align}
A_{wind,j,x+,d,t} &= (x_2 + x_3)(y_1 + y_2) - A_{oh0+}(-x_2 / 2, -y_2 / 2) - A_{sf0+}(-x_2 / 2, -y_2 / 2) \\
&- \{ (x_2 + x_3) y_1 - A_{oh0+}(-x_2 / 2, y_2 / 2) - A_{sf0+}(-x_2 / 2, y_2 / 2) \} \\
&- \{ x_3 (y_1 + y_2) - A_{oh0+}( x_2 / 2, -y_2 / 2) - A_{sf0+}( x_2 / 2, -y_2 / 2) \} \\
&+ x_3 y_1 - A_{oh0+}( x_2 / 2, y_2 / 2) - A_{sf0+}( x_2 / 2, y_2 / 2) \qquad \qquad \qquad (14) \\
\end{align} $$

In [ ]:
""" 式(14) Test """
# \確認.xlsx "式(14)Axp"シート → \Axp.csv を作成 → \TestConfig01 下に置いて読み込み → 計算

import Shading_Correction_Factor_Modules as SCFModule
import pandas as pd
import sys
csv_input = pd.read_csv(filepath_or_buffer="./TestConfig01/Axp.csv", encoding="ms932", sep=",")
if csv_input.columns[0]!="Axp_case":
    sys.exit("ファイル内に貼り付けたテスト条件が違います")
for i in range(len(csv_input)):
    case = csv_input.values[i][0]
    WSSize = csv_input.values[i][1:-3]
    [Azw, hs, AxpA] = csv_input.values[i][-3:]
    Axp = SCFModule.calc_Axp(WSSize, Azw, hs)

    print('case{}: Axp = {}, 期待値 = {}, 残差 = {}'.format( int(case), Axp, AxpA, Axp - AxpA ))

### B.4 太陽がx-側に位置する際のオーバーハングによる影の面積の算定式 (仕様書6.3.2 式(19))

$$
A_{oh0-}(x,y) = \left\{
\begin{array}{ll}
0 \hspace{24pt}(z_{y+}=0)
\\
\dfrac{1}{2} (x_{1y+} + x_2 / 2 + x) \dfrac{z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}}{z_{y+} \tan A_{ZW,j,d,t}} (x_{1y+} + x_2 / 2 + x)
\\
\hspace{30pt} \left( \begin{array}{ll}
x_{1y+} + x_2 / 2 + x < z_{y+} \tan A_{ZW,j,d,t} \\
y_{1} + y_2 / 2 - y \geq \dfrac{z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}}{z_{y+} \tan A_{ZW,j,d,t}} (x_{1y+} + x_2 / 2 + x) 
\end{array} \right)
\\
\Bigl\{ (x_{1y+} + x_2 / 2 + x) - \dfrac{1}{2} (y_{1} + y_2 / 2 - y) \dfrac{z_{y+} \tan A_{ZW,j,d,t}}{z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}} \Bigr\} (y_{1} + y_2 / 2 - y)
\\
\hspace{30pt} \left( \begin{array}{ll}
x_{1y+} + x_2 / 2 + x > \dfrac{z_{y+} \tan A_{ZW,j,d,t}}{z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}} (y_{1} + y_2 / 2 - y) \\
y_{1} + y_2 / 2 - y < z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}
\end{array} \right)
\\
( x_{1y+} + x_2 / 2 + x - \dfrac{1}{2} z_{y+} \tan A_{ZW,j,d,t} ) z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}
\\
\hspace{30pt} \left( \begin{array}{ll}
x_{1y+} + x_2 / 2 + x \geq z_{y+} \tan A_{ZW,j,d,t} \\
y_{1} + y_2 / 2 - y \geq z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}
\end{array} \right)
\end{array}
\right.  \qquad (19) 
$$

- コード中では、座標を入れ替えて、`calc_Aoh0p00`を叩くことで対応
  - 式$(15)$の変数 → 式$(19)$の変数
  - $x$ → $-x$
  - $x_{3y+}$ → $x_{1y+}$
  - $A_{ZW,j,d,t}$ → $-A_{ZW,j,d,t}$

In [ ]:
""" 式(19) Test """
# \確認.xlsx "式(19)Aoh0m"シート → \Aoh0m.csv を作成 → \TestConfig01 下に置いて読み込み → 計算

import Shading_Correction_Factor_Modules as SCFModule
import pandas as pd
import sys
csv_input = pd.read_csv(filepath_or_buffer="./TestConfig01/Aoh0m.csv", encoding="ms932", sep=",")
if csv_input.columns[0]!="Aoh0m_case":
    sys.exit("ファイル内に貼り付けたテスト条件が違います")
for i in range(len(csv_input)):
    [case, XX, YY]= csv_input.values[i][0:3]
    WSSize = csv_input.values[i][3:-3]
    [Azw, hs, Aoh0mA] = csv_input.values[i][-3:]
    Aoh0m = SCFModule.calc_Aoh0m(XX, YY, WSSize, Azw, hs)

    print('case{}: Aoh0m = {}, 期待値 = {}, 残差 = {}'.format( int(case), Aoh0m, Aoh0mA, Aoh0m - Aoh0mA ))

### B.5 太陽がx-側に位置する際のサイドフィンによる影の面積の算定式 (仕様書6.3.2 式(20))

$$
A_{sf0-}(x, y) = \left\{
\begin{array}{ll}
0 \hspace{24pt}(z_{x-}=0)
\\
\dfrac{1}{2} (y_{1x-} + y_2 / 2 - y) \dfrac{z_{x-} \tan A_{ZW,j,d,t}}{z_{x-} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}} (y_{1x-} + y_2 / 2 - y)
\\
\hspace{30pt} \left( \begin{array}{ll}
y_{1x-} + y_2 / 2 - y < z_{x-} \tan h_{S,d,t} / \cos A_{ZW,j,d,t} \\
x_{1} + x_2 / 2 + x \geq \dfrac{z_{x-} \tan A_{ZW,j,d,t}}{z_{x-} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}} (y_{1x-} + y_2 / 2 - y) 
\end{array} \right)
\\
\Bigl\{ (y_{1x-} + y_2 / 2 - y) - \dfrac{1}{2} (x_{1} + x_2 / 2 + x) \dfrac{z_{x-} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}}{z_{x-} \tan A_{ZW,j,d,t} } \Bigr\} (x_{1} + x_2 / 2 + x)
\\
\hspace{30pt} \left( \begin{array}{ll}
y_{1x-} + y_2 / 2 - y > \dfrac{z_{x-} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}}{z_{x-} \tan A_{ZW,j,d,t}} (x_{1} + x_2 / 2 + x) \\
x_{1} + x_2 / 2 + x < z_{x-} \tan A_{ZW,j,d,t}
\end{array} \right)
\\
( y_{1x-} + y_2 / 2 - y - \dfrac{1}{2} z_{x-} \tan h_{S,d,t} / \cos A_{ZW,j,d,t} ) \; z_{x-} \tan A_{ZW,j,d,t}
\\
\hspace{30pt} \left( \begin{array}{ll}
y_{1x-} + y_2 / 2 - y \geq z_{x-} \tan h_{S,d,t} / \cos A_{ZW,j,d,t} \\
x_{1} + x_2 / 2 + x \geq z_{x-} \tan A_{ZW,j,d,t}
\end{array} \right)
\end{array}
\right.  \qquad (20) 
$$

- コード中では、座標を入れ替えて、`calc_Aoh0p00`を叩くことで対応
  - 式$(15)$の変数 → 式$(16)$の変数 → 式$(20)$の変数
  - $x$            → $y$            → $y$
  - $x_2$          → $y_2$          → $y_2$
  - $x_{3y+}$      → $y_{1x+}$      → $y_{1x-}$
  - $y$            → $x$            → $-x$
  - $y_1$          → $x_3$          → $x_1$
  - $y_2$          → $x_2$          → $x_2$
  - $z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}$ → $z_{x+} \tan | A_{ZW,j,d,t} |$ → $z_{x-} \tan A_{ZW,j,d,t}$
  - $z_{y+} \tan | A_{ZW,j,d,t} |$ → $z_{x+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}$ → $z_{x-} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}$

In [ ]:
""" 式(20) Test """
# \確認.xlsx "式(20)Asf0m"シート → \Asf0m.csv を作成 → \TestConfig01 下に置いて読み込み → 計算

import Shading_Correction_Factor_Modules as SCFModule
import pandas as pd
import sys
csv_input = pd.read_csv(filepath_or_buffer="./TestConfig01/Asf0m.csv", encoding="ms932", sep=",")
if csv_input.columns[0]!="Asf0m_case":
    sys.exit("ファイル内に貼り付けたテスト条件が違います")
for i in range(len(csv_input)):
    [case, XX, YY]= csv_input.values[i][0:3]
    WSSize = csv_input.values[i][3:-3]
    [Azw, hs, Asf0mA] = csv_input.values[i][-3:]
    Asf0m = SCFModule.calc_Asf0m(XX, YY, WSSize, Azw, hs)

    print('case{}: Asfom = {}, 期待値 = {}, 残差 = {}'.format( int(case), Asf0m, Asf0mA, Asf0m - Asf0mA ))

### B.6 太陽がx-側に位置する際の日射が射す部分の面積の計算式 (仕様書6.3.2 式(18))

$$ \begin{align}
A_{wind,j,x-,d,t} &= (x_1 + x_2)(y_1 + y_2) - A_{oh0-}( x_2 / 2, -y_2 / 2) - A_{sf0-}( x_2 / 2, -y_2 / 2) \\
&- \{ (x_1 + x_2) y_1 - A_{oh0-}( x_2 / 2, y_2 / 2) - A_{sf0-}( x_2 / 2, y_2 / 2) \} \\
&- \{ x_1 (y_1 + y_2) - A_{oh0-}(-x_2 / 2, -y_2 / 2) - A_{sf0-}(-x_2 / 2, -y_2 / 2) \} \\
&+ x_1 y_1 - A_{oh0-}(-x_2 / 2, y_2 / 2) - A_{sf0-}(-x_2 / 2, y_2 / 2) \qquad \qquad \qquad (18) \\
\end{align} $$

In [ ]:
""" 式(18) Test """
# \確認.xlsx "式(18)Axm"シート → \Axm.csv を作成 → \TestConfig01 下に置いて読み込み → 計算

import Shading_Correction_Factor_Modules as SCFModule
import pandas as pd
import sys
csv_input = pd.read_csv(filepath_or_buffer="./TestConfig01/Axm.csv", encoding="ms932", sep=",")
if csv_input.columns[0]!="Axm_case":
    sys.exit("ファイル内に貼り付けたテスト条件が違います")
for i in range(len(csv_input)):
    case = csv_input.values[i][0]
    WSSize = csv_input.values[i][1:-3]
    [Azw, hs, AxmA] = csv_input.values[i][-3:]
    Axm = SCFModule.calc_Axm(WSSize, Azw, hs)

    print('case{}: Axm = {}, 期待値 = {}, 残差 = {}'.format( int(case), Axm, AxmA, Axm - AxmA ))

### B.7 直達日射が窓に射す部分の面積の計算

- 太陽高度$h_{S,d,t}[deg]$が、$h_{S,d,t}>0°$で計算
- 窓面の法線ベクトルと太陽位置とのなす水平面上の角度$A_{ZW,j,d,t}$に応じて以下。
  - $-90°<A_{ZW,j,d,t} < 0°$：$A_{wind,j,x-,d,t}$を計算($A_{wind,j,x+,d,t} = 0$)
  - $0°\leq A_{ZW,j,d,t} < 90°$：$A_{wind,j,x+,d,t}$を計算($A_{wind,j,x-,d,t} = 0$)


## C. 天空日射・反射日射の効果係数 (仕様書6.4, 6.5)

### C.1 形態係数算定のための関数$\;f_A\;$ (仕様書6.4 式(23))

$$ \begin{align}
f_{A}(x_a,x_b,y_a,y_b,z_a) &= \frac{x_b \sqrt{y_b^2+z_a^2}}{2} \tan^{-1} \frac{x_b}{\sqrt{y_b^2+z_a^2}}
- \frac{x_b \sqrt{y_a^2+z_a^2}}{2} \tan^{-1} \frac{x_b}{\sqrt{y_a^2+z_a^2}} \\
&- \frac{x_a \sqrt{y_b^2+z_a^2}}{2} \tan^{-1} \frac{x_a}{\sqrt{y_b^2+z_a^2}}
+ \frac{x_a \sqrt{y_a^2+z_a^2}}{2} \tan^{-1} \frac{x_a}{\sqrt{y_a^2+z_a^2}} \\
&+ \frac{x_b^2 - y_b^2 - z_a^2}{8} \log (x_b^2 + y_b^2 + z_a^2) - \frac{x_b^2 - y_a^2 - z_a^2}{8} \log (x_b^2 + y_a^2 + z_a^2) \\
&- \frac{x_a^2 - y_b^2 - z_a^2}{8} \log (x_a^2 + y_b^2 + z_a^2) + \frac{x_a^2 - y_a^2 - z_a^2}{8} \log (x_a^2 + y_a^2 + z_a^2) 
\qquad \qquad \qquad (23) \\
\end{align} $$

In [ ]:
""" 式(23) Test """
# \確認.xlsx "式(23)fA"シート → \fA.csv を作成 → \TestConfig01 下に置いて読み込み → 計算

import Shading_Correction_Factor_Modules as SCFModule
import pandas as pd
import sys
csv_input = pd.read_csv(filepath_or_buffer="./TestConfig01/fA.csv", encoding="ms932", sep=",")
if csv_input.columns[0]!="fA_case":
    sys.exit("ファイル内に貼り付けたテスト条件が違います")
for i in range(len(csv_input)):
    [case, xa, xb, ya, yb, za, fAA] = csv_input.values[i]
    fA = SCFModule.calc_fa(xa, xb, ya, yb, za)
    print('case{}: fA = {}, 期待値 = {}, 残差 = {}'.format( int(case), fA, fAA, fA - fAA ))

### C.2 天空に対する形態係数 (仕様書6.4 式(22))

$$ \begin{align}
\phi_{j,y+} = (\pi A_{wind,j})^{-1} \{ & \hspace{2pt} f_A(x_{3y+}, x_2 + x_{3y+}, y_1, y_1 + y_2, z_{y+}) + f_A(y_{1x+}, y_{1x+} + y_2, x_{3}, x_2 + x_{3}, z_{x+}) \\
+ & \hspace{2pt} f_A(x_{1y+}, x_{1y+} + x_2, y_1, y_1 + y_2, z_{y+}) + f_A(y_{1x-}, y_{1x-} + y_2, x_{1}, x_1 + x_2, z_{x-}) \\ 
+ & \hspace{2pt} f_A(x_{3}, x_2 + x_{3}, y_1, y_1 + y_2, 0) + f_A(y_{1}, y_{1} + y_2, x_{3}, x_2 + x_{3}, 0) \\
+ & \hspace{2pt} f_A(x_{1}, x_{1} + x_2, y_1, y_1 + y_2, 0) + f_A(y_{1}, y_{1} + y_2, x_{1}, x_1 + x_2, 0) \\ 
- & \hspace{2pt} f_A(x_{3y+}, x_2 + x_{3y+}, y_1, y_1 + y_2, 0) - f_A(y_{1x+}, y_{1x+} + y_2, x_{3}, x_2 + x_{3}, 0) \\
- & \hspace{2pt} f_A(x_{1y+}, x_{1y+} + x_2, y_1, y_1 + y_2, 0) - f_A(y_{1x-}, y_{1x-} + y_2, x_{1}, x_1 + x_2, 0) \qquad (22) \\
\end{align} $$

In [ ]:
""" 式(22) Test """
# \確認.xlsx "式(22)phiyp"シート → \phiyp.csv を作成 → \TestConfig01 下に置いて読み込み → 計算

import Shading_Correction_Factor_Modules as SCFModule
import pandas as pd
import sys
csv_input = pd.read_csv(filepath_or_buffer="./TestConfig01/phiyp.csv", encoding="ms932", sep=",")
if csv_input.columns[0]!="phiyp_case":
    sys.exit("ファイル内に貼り付けたテスト条件が違います")
for i in range(len(csv_input)):
    case = csv_input.values[i][0]
    WSSize = csv_input.values[i][1:-1]
    phiypA = csv_input.values[i][-1]
    phiyp = SCFModule.calc_phiyp(WSSize)

    print('case{}: phiyp = {}, 期待値 = {}, 残差 = {}'.format( int(case), phiyp, phiypA, phiyp - phiypA ))

### C.3 天空日射の効果係数 (仕様書6.4 式(21))

$$\gamma_{isr,j,y+} = 2 \phi_{j,y+} \qquad (21) $$

### C.4 地面に対する形態係数 (仕様書6.5 式(25))

$$ \begin{align}
\phi_{j,y-} = (\pi A_{wind,j})^{-1} \{ & \hspace{2pt} f_A(x_{3y-}, x_2 + x_{3y-}, y_3, y_2 + y_3, z_{y-}) + f_A(y_{3x+}, y_2 + y_{3x+}, x_{3}, x_2 + x_{3}, z_{x+}) \\
+ & \hspace{2pt} f_A(x_{1y-}, x_{1y-} + x_2, y_3, y_2 + y_3, z_{y-}) + f_A(y_{3x-}, y_2 + y_{3x-}, x_{1}, x_1 + x_2, z_{x-}) \\ 
+ & \hspace{2pt} f_A(x_{3}, x_2 + x_{3}, y_3, y_2 + y_3, 0) + f_A(y_{3}, y_{2} + y_3, x_{3}, x_2 + x_{3}, 0) \\
+ & \hspace{2pt} f_A(x_{1}, x_{1} + x_2, y_3, y_2 + y_3, 0) + f_A(y_{3}, y_{2} + y_3, x_{1}, x_1 + x_2, 0) \\ 
- & \hspace{2pt} f_A(x_{3y-}, x_2 + x_{3y-}, y_3, y_2 + y_3, 0) - f_A(y_{3x+}, y_2 + y_{3x+}, x_{3}, x_2 + x_{3}, 0) \\
- & \hspace{2pt} f_A(x_{1y-}, x_{1y-} + x_2, y_3, y_2 + y_3, 0) - f_A(y_{3x-}, y_2 + y_{3x-}, x_{1}, x_1 + x_2, 0) \qquad (25) \\
\end{align} $$

In [ ]:
""" 式(25) Test """
# \確認.xlsx "式(25)phiym"シート → \phiym.csv を作成 → \TestConfig01 下に置いて読み込み → 計算

import Shading_Correction_Factor_Modules as SCFModule
import pandas as pd
import sys
csv_input = pd.read_csv(filepath_or_buffer="./TestConfig01/phiym.csv", encoding="ms932", sep=",")
if csv_input.columns[0]!="phiym_case":
    sys.exit("ファイル内に貼り付けたテスト条件が違います")
for i in range(len(csv_input)):
    case = csv_input.values[i][0]
    WSSize = csv_input.values[i][1:-1]
    phiymA = csv_input.values[i][-1]
    phiym = SCFModule.calc_phiym(WSSize)

    print('case{}: phiym = {}, 期待値 = {}, 残差 = {}'.format( int(case), phiym, phiymA, phiym - phiymA ))

### C.5 反射日射の効果係数 (仕様書6.5 式(24))

$$\gamma_{isr,j,y-} = 2 \phi_{j,y-} \qquad (24) $$

## D. 地点と日射量


### D.3 正時±30分で太陽が地平線上にある時間刻み数のカウント (仕様書6.2 式(3)及び図5中の$n_H$の計算)

- 算定ツール標準の時間分割数$n_{\Delta t}$は、$6$ ($10$分刻み)
  - $1$時間を$1$分割もしくは$2$以上の偶数で分割する
  
  
- 正時$\pm 30$分間で太陽が地平線上にある(太陽高度$>0$)時間刻み数をカウントして、$n_H$を計算

In [ ]:
""" 式(3),図5中のNhの計算 Test """
# \確認.xlsx "式(3)中Nh"シート → \Nh.csv を作成 → \TestConfig01 下に置いて読み込み → 計算

import Shading_Correction_Factor_Modules as SCFModule
import pandas as pd
import sys
csv_input = pd.read_csv(filepath_or_buffer="./TestConfig01/Nh.csv", encoding="ms932", sep=",")
if csv_input.columns[0]!="Nh_case":
    sys.exit("ファイル内に貼り付けたテスト条件が違います")
for i in range(len(csv_input)):
    [case, Latitude, Longitude, NDay, NHour, NDT, NhA] = csv_input.values[i]  
    Nh = SCFModule.calc_Nh(Latitude, Longitude, NDay, NHour, NDT)
    print('case{}: Nh = {}, 期待値 = {}, 残差 = {}'.format( int(case), Nh, NhA, Nh - NhA ))

print ('NDT = {} のときには Nh = {} か {}'.format( 1, SCFModule.calc_Nh(35, 135, 121, 6, 1)
                                            , SCFModule.calc_Nh(35, 135, 121, 5, 1) ))
 
print(SCFModule.calc_Nh(35, 135, 121, 6, 5)) #奇数分割時のエラー確認

### D.4 $1/n_{\Delta t}$時間間隔での日射量 (仕様書6.2 式(3)の計算, 図5参照)

- 算定ツール標準の時間分割数$n_{\Delta t}$は、$6$ ($10$分刻み)
- 正時$\pm 30$分間で太陽が地平線上にある(太陽高度$>0$)時間刻み数をカウントして、$n_H$を計算
- 法線面直達日射量, 水平面天空日射量をそれぞれに適用し、時間刻みにおける日射量を算定する

In [4]:
""" 式(3)の S'HM の計算 Test """
# \確認.xlsx "式(3)中Nh"シート → \Sdhm.csv を作成 → \TestConfig01 下に置いて読み込み → 計算

import nbimporter
import Shading_Correction_Factor_Modules as SCFModule
import pandas as pd
import sys
csv_input = pd.read_csv(filepath_or_buffer="./TestConfig01/Sdhm.csv", encoding="ms932", sep=",")
if csv_input.columns[0]!="Sdhm_case":
    sys.exit("ファイル内に貼り付けたテスト条件が違います")
for i in range(len(csv_input)):
    [case, MM, NDT, sinh, Sh, Shp, Nh, Nhp, SdhmA] = csv_input.values[i]  
    Sdhm = SCFModule.calc_Sdhm(MM, NDT, sinh, Sh, Shp, Nh, Nhp)
    print('case{}: Sdhm = {}, 期待値 = {}, 残差 = {}'.format( int(case), Sdhm, SdhmA, Sdhm - SdhmA ))

Importing Jupyter notebook from Shading_Correction_Factor_Modules.ipynb
Importing Jupyter notebook from climate.ipynb
case1: Sdhm = 0, 期待値 = 0.0, 残差 = 0.0
case2: Sdhm = 0, 期待値 = 0.0, 残差 = 0.0
case3: Sdhm = 0, 期待値 = 0.0, 残差 = 0.0
case4: Sdhm = 0, 期待値 = 0.0, 残差 = 0.0
case5: Sdhm = 0, 期待値 = 0.0, 残差 = 0.0
case6: Sdhm = 0, 期待値 = 0.0, 残差 = 0.0
case7: Sdhm = 0, 期待値 = 0.0, 残差 = 0.0
case8: Sdhm = 0, 期待値 = 0.0, 残差 = 0.0
case9: Sdhm = 0, 期待値 = 0.0, 残差 = 0.0
case10: Sdhm = 0, 期待値 = 0.0, 残差 = 0.0
case11: Sdhm = 0, 期待値 = 0.0, 残差 = 0.0
case12: Sdhm = 0, 期待値 = 0.0, 残差 = 0.0
case13: Sdhm = 0, 期待値 = 0.0, 残差 = 0.0
case14: Sdhm = 0, 期待値 = 0.0, 残差 = 0.0
case15: Sdhm = 0, 期待値 = 0.0, 残差 = 0.0
case16: Sdhm = 0, 期待値 = 0.0, 残差 = 0.0
case17: Sdhm = 0, 期待値 = 0.0, 残差 = 0.0
case18: Sdhm = 0, 期待値 = 0.0, 残差 = 0.0
case19: Sdhm = 0.5714285714285714, 期待値 = 0.5714285714285711, 残差 = 3.3306690738754696e-16
case20: Sdhm = 0.5714285714285714, 期待値 = 0.5714285714285711, 残差 = 3.3306690738754696e-16
case21: Sdhm = 0.571428571428

### D.5 窓ガラスの入射角特性読み込み

- `\IncidentAngleCharacteristics.csv`
  - $1$行目はヘッダ："入射角特性", $\eta_{max}$, $\eta_{isr}$, $k_0$～$k_7$
  - $2$行目以下に入射角特性のデータを記入
    - $1$列：$ID$
    - $2$列：直達日射に対する入射角特性最大値(入射角$0$) $\eta_{max}$
    - $3$列：天空・反射日射に対する入射角特性(遮蔽なしの場合) $\eta_{isr}$
    - $4～11$列：$\eta_{j,d,t}(\theta_{j,d,t})$ 算定式の係数$k_n$$(n=0～7)$
       $$\eta_{j,d,t}(\theta_{j,d,t}) = \sum_{n=0}^7 k_n \cos^n \theta_{j,d,t}$$
  - デフォルトとして、以下を設定
    - $ID=0$：日よけ効果係数内で入射角特性非考慮 → $\eta_{j,d,t}(\theta_{j,d,t}) = 1$
    - $ID=1$：解説書の入射角特性(「平成25年度省エネルギー基準に準拠した算定・判断の方法及び解説 I 非住宅建築物 第二版(連合印刷センター, 平成26年○月○日)」, pp.168-170, 式(2.1.25),(2.1.28),(2.1.32))
       $$\eta_{j,d,t}(\theta_{j,d,t}) = 2.3920 \cos \theta_{j,d,t} -3.8636 \cos^3 \theta_{j,d,t} + 3.7568 \cos^5 \theta_{j,d,t} - 1.3952 \cos^7 \theta_{j,d,t} $$
    - 他の特性を入れる際には、$ID$を違えて、`\IncidentAngleCharacteristics.csv` に追加する。

In [ ]:
""" \IncidentAngleCharacteristics.csv の読み込みテスト """

import Shading_Correction_Factor_Modules as SCFModule
print(SCFModule.input_IncidentAngleCharacteristics(0, "./SCFConfig01/", "IncidentAngleCharacteristics.csv"))
print(SCFModule.input_IncidentAngleCharacteristics(1, "./SCFConfig01/", "IncidentAngleCharacteristics.csv"))
print(SCFModule.input_IncidentAngleCharacteristics(2, "./SCFConfig01/", "IncidentAngleCharacteristics.csv"))

### D.6 直達日射に対する窓ガラスの入射角特性 (緑本非住宅第二版pp.169 式(2.1.28)準拠)

- 「平成25年度省エネルギー基準に準拠した算定・判断の方法及び解説 I 非住宅建築物 第二版(連合印刷センター, 平成26年○月○日)」pp.168-170参照
- 入射角$\theta_{j,d,t}[deg]$, 太陽高度$h_{S,d,t}[deg]$, 太陽方位角$A_{ZS,d,t}[deg]$, 外壁$j$の方位角$A_{ZW,i}[deg]$
- 入射角特性算定式の係数$k_n$$(n=0～7)$, 日付$d$時刻$t$における入射角特性$\eta_{j,d,t}$

$$\cos \theta_{j,d,t} = \cos h_{S,d,t} \cos (A_{ZS,d,t} - A_{ZW,i}) \qquad (2.1.26) $$
$$\eta_{j,d,t}(\theta_{j,d,t}) = \sum_{n=0}^7 k_n \cos^n \theta_{j,d,t} \qquad (2.1.28') $$

In [ ]:
""" 式(2.1.26),(2.1.28')の計算 Test """
# \確認.xlsx "式(1)Azwjdt"シート → \etajdt.csv を作成 → \TestConfig01 下に置いて読み込み → 計算

import Shading_Correction_Factor_Modules as SCFModule
import pandas as pd
import sys
csv_input = pd.read_csv(filepath_or_buffer="./TestConfig01/etajdt.csv", encoding="ms932", sep=",")
if csv_input.columns[0]!="etajdt_case":
    sys.exit("ファイル内に貼り付けたテスト条件が違います")

for i in range(len(csv_input)):
    [case, Azwjdt, cosh] = csv_input.values[i][0:3]  
    etakk = csv_input.values[i][3:11]
    etajdtA = csv_input.values[i][-1] 
    costheta = SCFModule.calc_costheta(Azwjdt, cosh)
    etajdt = SCFModule.calc_etajdt(costheta, etakk)
    print('case{}: etajdt = {}, 期待値 = {}, 残差 = {}'.format( int(case), etajdt, etajdtA, etajdt - etajdtA ))


## E. 日よけ効果係数

### E.1 日よけ効果係数の算出 (仕様書6.1 式(2)準拠)

- $\gamma_{dsr,j,x+}$：窓等$j$に対して太陽が$x+$側に位置する際の直達日射に対する日よけ効果係数(式$(13)$)
- $\gamma_{dsr,j,x-}$：窓等$j$に対して太陽が$x-$側に位置する際の直達日射に対する日よけ効果係数(式$(17)$)
- $\gamma_{isr,j,y+}$：天空日射に対する日よけ効果係数(式$(21)$) → C.3参照
- $\gamma_{isr,j,y-}$：反射日射に対する日よけ効果係数(式$(24)$) → C.5参照
- $\eta_{j,d,t}$：日付$d$、時刻$t$における窓等$j$の入射角特性 → D.6参照
- $S_{D,d,t}$：日付$d$、時刻$t$における法線面直達日射量$[kcal/m^2]$
- $S_{S,d,t}$：日付$d$、時刻$t$における水平面天空日射量$[kcal/m^2]$
- $\eta_{isr}$：天空・反射日射に対する入射角特性(遮蔽なしの場合)$[-]$
  - 緑本、仕様書では$\;\eta_{isr} = 0.808\;$
  
  
- 式(2)のうち、定数$0.5$は垂直面からみた天空・地表面の形態係数、$0.1$は地表面における日射反射率である。

$$ \begin{align}
\gamma_{wind,j} 
= \big[&\hspace{4pt}\gamma_{dsr,j,x+} \times \sum_{-90<A_{ZW,j,d,t}<0, \; h_{s,d,t}>0} 
        (S_{D,d,t} \; \eta_{j,d,t} \cos h_{s,d,t} \cos A_{ZW,j,d,t}) \\
  + &\hspace{4pt} \gamma_{dsr,j,x-} \times \sum_{0 \leq A_{ZW,j,d,t}<90, \; h_{s,d,t}>0} 
        (S_{D,d,t} \; \eta_{j,d,t} \cos h_{s,d,t} \cos A_{ZW,j,d,t}) \\
  + &\hspace{4pt}\gamma_{isr,j,y+} \times \sum (\eta_{isr} \times 0.5 \times S_{S,d,t} \\
  + &\hspace{4pt}\gamma_{isr,j,y-} \times \sum \{\eta_{isr} \times 0.1 \times 0.5 
        \times (S_{S,d,t} + S_{D,d,t} \sin h_{s,d,t}) \} \big] \\
\big/ \hspace{4pt} \big[&\sum_{-90<A_{ZW,j,d,t}<90, \; h_{s,d,t}>0} (S_{D,d,t} \; \eta_{j,d,t} \cos h_{s,d,t} \cos A_{ZW,j,d,t}) \\
    + &\hspace{4pt}\sum \{\eta_{isr} \times 0.5 \times S_{S,d,t} 
        +  \eta_{isr} \times 0.1 \times 0.5 \times (S_{S,d,t} + S_{D,d,t} \sin h_{s,d,t}) \} \big] \qquad (2)
\\
\end{align} $$

- $A_{wind,j}$：窓等$j$の面積($=x_2y_2$)$[m^2]$
- $A_{wind,j,x+,d,t}$：太陽が$x+$側に位置する日付$d$、時刻$t$において窓等$j$の直達日射が当たる部分の面積$[m^2]$
- $A_{wind,j,x-,d,t}$：太陽が$x-$側に位置する日付$d$、時刻$t$において窓等$j$の直達日射が当たる部分の面積$[m^2]$

$$ \begin{align}
\gamma_{dsr,j,x+} 
= \big\{ & \sum_{-90<A_{ZW,j,d,t}<0, \; h_{s,d,t}>0} (A_{wind,j,x+,d,t}\hspace{2pt}  
        S_{D,d,t} \; \eta_{j,d,t} \cos h_{s,d,t} \cos A_{ZW,j,d,t}) \big\} \\
\big/ \big\{ &\hspace{2pt} A_{wind,j} \sum_{-90<A_{ZW,j,d,t}<0, \; h_{s,d,t}>0} (S_{D,d,t} \; \eta_{j,d,t} \cos h_{s,d,t} \cos A_{ZW,j,d,t})  \} \big\} \qquad (13)
\\
\end{align} $$

$$ \begin{align}
\gamma_{dsr,j,x-} 
= \big\{ & \sum_{0 \leq A_{ZW,j,d,t}<90, \; h_{s,d,t}>0} (A_{wind,j,x-,d,t} \hspace{2pt} 
        S_{D,d,t} \; \eta_{j,d,t} \cos h_{s,d,t} \cos A_{ZW,j,d,t}) \big\} \\
\big/ \big\{ &\hspace{2pt} A_{wind,j} \sum_{0 \leq A_{ZW,j,d,t}<90, \; h_{s,d,t}>0} (S_{D,d,t} \; \eta_{j,d,t} \cos h_{s,d,t} \cos A_{ZW,j,d,t})  \} \big\} \qquad (17)
\\
\end{align} $$



### E.2 式(2)分子分母中の時刻$t$における各成分の計算


### E.3 式(2)分子分母の期間積算処理

- 式(2)中の時刻$t$における分子および分母を期間積算する。
  - 暖房期、冷房期、非空調期
  - 各月毎
  - 各月、各時刻毎
    - 時刻毎は正時$\pm30$分としたいが、本ツール内でのデータの取り扱いからずれる。前時刻の`MM` $=$ `NDT`$/2～$同時刻の`MM` $=$ `NDT`$/2-1$の間を正時の積算値として計上する。
    

- `SCF[h][i][j]`に格納して積算
  - `h`$=0$：分子, `h`$=1$：分母, `h`$=2$：効果係数
  - `i`$=-2$：冷房期積算, `i`$=-1$：暖房期積算, `i`$=0$：非空調期積算, `i`$=1～12$：各月で積算
  - `j`$=-1$：日積算, `j`$=0～24$：各時刻で積算  

  



## X. 日よけ効果係数算定ツール本体＋入出力



### X.2 本体プログラムテスト

#### X.2.1 まずは動くことを確認

In [ ]:
""" 日よけ効果係数計算プログラム本体 Test """

import Shading_Correction_Factor_Main as SCFMain

Path00 = "./SCFConfig01/"  # 設定ファイルのあるパス
FileName00 = "Zone.csv"    # 地点データのファイル
FileName01 = "IncidentAngleCharacteristics.csv"   # 窓ガラスの入射角特性ファイル
ClimateZone = 6            # 地域区分(1～8地域, 他、ユニークなID設定可)
NDT = 6                    # 1時間の分割数,ツールの標準は6分割
Azimuth = "南"             # 窓面の方位を16方位か、角度(-180°< Azimuth <= 180°)で入力
etaID = 1                  # 入射角特性のデータセットのID
WSSize = [1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.24, 0.28, 0.21, 0.2]
    # WSSize = [X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym]
    # WSSize:窓および日よけの寸法一式

SCF = SCFMain.Calc_ShadingCorrectionFactor(Path00, FileName00, FileName01
                                           , ClimateZone, NDT, etaID, Azimuth, WSSize )
                 
    # SCF[h][i][j]
    # h = 0：分子, h = 1：分母, h = 2：効果係数
    # i = -2：冷房期積算, i = -1：暖房期積算, i = 0：非空調期積算,O
    #         i = 1～12：各月で積算
    # j = -1：日積算, j = 0～24：各時刻で積算             

print('冷房期 = {}, 暖房期 = {}, 非空調期 = {}'.format( SCF[2][-2][-1], SCF[2][-1][-1], SCF[2][0][-1] ))
for i in range(1,13):
    for j in range(25):
        print(i,j,SCF[0][i][j],SCF[1][i][j],SCF[2][i][j])

#### X.2.2 本体プログラム計算過程デバッグ用

In [ ]:
""" 日よけ効果係数計算プログラム本体 デバッグ用"""
# ●● デバッグ用に本体プログラムをコピーして確認('180503) ●●
# ●● ←付きコメント分はデバッグ作業時のメモ ●●

import sys    # ●● Exit用に読み込み追加 ●●
import Shading_Correction_Factor_Modules as SCFModule
import nbimporter
import climate

# def Calc_ShadingCorrectionFactor(Path00, FileName00, FileName01, ClimateZone, NDT, etaID, Azimuth, WSSize ):
# ●● ↑デバッグ用にコメントアウト ●●

for iii in range(1):   # ●● ←デバッグ用ダミー ●●

    
    # ●● ↓デバッグ用に本来の引数を直接指定 ●●
    """ 引数(の例) """    
    Path00 = "./SCFConfig01/"  # 設定ファイルのあるパス
    FileName00 = "Zone.csv"    # 地点データのファイル
    FileName01 = "IncidentAngleCharacteristics.csv"   # 窓ガラスの入射角特性ファイル
    ClimateZone = 3            # 地域区分(1～8地域, 他、ユニークなID設定可)
    # ●● ↑ 検証用の \日除け効果係数算定シート(全対応)-9.xlsm は、3地域盛岡指定 ●●
    NDT = 6                    # 1時間の分割数,ツールの標準は6分割
    Azimuth = "南"             # 窓面の方位を16方位か、角度(-180°< zimuth <= 180°)で入力
    etaID = 1                  # 入射角特性のデータセットのID
    WSSize = [0.2,2, 0.2,0.2, 0.2, 0.2, 0.2, 0.2,2, 0.2,0.2, 0.2, 0.2, 0.2, 1,  0.5,1,  0.5]
        # ●● ↑ 検証用の \日除け効果係数算定シート(全対応)-9.xlsm の設定 ●●
    # WSSize = [X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym]
    # WSSize:窓および日よけの寸法一式
    # ●● ↓ 検証用の \日除け効果係数算定シート(全対応)-9.xlsm の設定の確認 ●●
    # 0.2,2,0.2,0.2,0.2,0.2,0.2,0.2,2,0.2,0.2,0.2,0.2,0.2,1,0.5,1,0.5
    # X1, X2,X3, X1yp,X1ym,X3yp,X3ym,Y1, Y2,Y3, Y1xp,Y1xm,Y3xp,Y3xm,Zxp,Zxm,Zyp,Zym
    # 0.2,2, 0.2,0.2, 0.2, 0.2, 0.2, 0.2,2, 0.2,0.2, 0.2, 0.2, 0.2, 1,  0.5,1,  0.5
    
    
    """ リストの初期設定 """    
    Nh = [0 for i in range(8761)]
    SCF01 = [[[0 for j in range(NDT)] for i in range(8761)] for h in range(2)]
    
    """ \Zone.csv から地点データの読み込み(D.1) """
        # Latitude:緯度, Longitude:経度
    c = climate.Climate(ClimateZone)

    Latitude  = c.latitude
    Longitude = c.longitude
        
    """ 気象データ読み込み(D.2) """
        # SRHour[i][j]：1時間間隔データ
        #               i=0：1/1 0時 ～ 8760：12/31 24時
        #               j=0：月日時刻の5桁or6桁表記, j=1：法線面直達日射量[kcal/(m2h)]
        #               j=2：水平面天空日射量[kcal/(m2h)], j=3：暖房冷房判定タグ(暖房期:1, 冷房期:2, 非空調期:0)
    SRHour = c.SRHour
    
    """ 窓面の方位(A.8) """
        # Azwj:窓面の方位(-180°< Azwj <= 180°)
    Azwj = SCFModule.calc_Azwj(Azimuth)
    # print(Azwj)  # ●● ←デバッグ用
        # ●● → 出力：0.0       → OK
    # sys.exit("")      # ●● ←デバッグ用
    
    """ 天空日射の効果係数(C.3) """    
        # gammayp:天空日射の効果係数, 天空の形態係数の2倍
    gammayp = SCFModule.calc_gammayp(WSSize)

    """ 反射日射の効果係数(C.5) """    
        # gammaym:反射日射の効果係数, 地面の形態係数の2倍
    gammaym = SCFModule.calc_gammaym(WSSize)
    # print(gammayp, gammaym)  # ●● ←デバッグ用
        # ●● → 出力：0.583931809607 0.701354727231 
        # ●● \日除け効果係数算定シート(全対応)-9.xlsm「算定」シート セルKM8,KN9 より、
        # ●●        「0.58393181」 「0.701354727」      → OK
    # sys.exit("")      # ●● ←デバッグ用

    """ 窓ガラスの入射角特性読み込み(D.5) """  
        # etamax:直達日射に対する入射角特性最大値(入射角0)
        # etaisr:天空・反射日射に対する入射角特性(遮蔽なしの場合)
        # etakk:入射角特性の係数, (cosθ)^0 ～ (cosθ)^7 の各項の係数 
    [etaID0, etamax, etaisr, etakk] = SCFModule.input_IncidentAngleCharacteristics(etaID, Path00, FileName01)    
    # print(etaID0, etamax, etaisr, etakk)  # ●● ←デバッグ用
        # ●● → 出力：1.0 0.88 0.808 [ 0.      2.392   0.     -3.8636  0.      3.7568  0.     -1.3952]
        # ●● \日除け効果係数算定シート(全対応)-9.xlsm「算定」シート セルM19 の計算式より、
        # ●● cosθの係数：2.392, (cosθ)^3の係数：-3.8636, (cosθ)^5の係数：3.7568, (cosθ)^7の係数：-1.3952 
        # ●● → OK
    # sys.exit("")      # ●● ←デバッグ用
    
    """ 窓面積算定 """  
        # Awj:窓面積[m2]
    Awj = WSSize[1] * WSSize[8]    # Awj = X2 * Y2
    # print(Awj)  # ●● ←デバッグ用
        # ●● → 出力：4
        # ●● X2 = 2, Y2 = 2 よりOK
    # sys.exit("")      # ●● ←デバッグ用
    
    """ +++++++++++++++++++++++++++++++++++ 1時間のループ【1回目】→ Nhを通しで計算するためだけ ++++ """       
    for Hour00 in range(8761):
        """ 日時の計算(A.1)【1回目】 """
            # Hour00:1年間の通しの時刻, 1/1 0時がHour00=0, 12/31 24時がHour00=8760
            # NDay:1/1を"1", 12/31を"365"とする年頭からの通しの日数
            # NHour:時刻(0時～23時),12/31のみ24時あり
        [NDay,NHour] = SCFModule.calc_NDayNHour(Hour00)
        # print(Hour00,NDay,NHour)  # ●● ←デバッグ用
            # ●● → \確認(全体動作).xlsx「NDayNhour」シートで確認
            # ●● → OK

        """ 正時±30分で太陽が地平線上にある時間刻み数のカウント(D.3) """
            # Nh[Hour00]:正時±30分で太陽が地平線上にある時間刻み数のカウント数
            # NDT:1時間の分割数,ツールの標準は6分割
        Nh[Hour00] = SCFModule.calc_Nh(Latitude, Longitude, NDay, NHour, NDT)
        # print(Hour00,Nh[Hour00])  # ●● ←デバッグ用
            # ●● → \確認(全体動作).xlsx「Nh」シートで確認
            # ●● → OK        
        
    # sys.exit("")      # ●● ←デバッグ用
        
    """ ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ 1時間のループ【2回目】 ++++ """       
    for Hour00 in range(8760):      # ← 12/31 24時は回さないのでHour00=8759がループの最後
        """ 日時の計算(A.1)【2回目】 """
        [NDay,NHour] = SCFModule.calc_NDayNHour(Hour00)    
            # ●● 上で確認済み
        
        """ 赤緯の計算(A.2) """
            # delta_d:赤緯[deg]
        deltad = SCFModule.calc_deltad(NDay)
        # print(Hour00,deltad)  # ●● ←デバッグ用
            # ●● → \確認(全体動作).xlsx「deltad」シートで確認
            # ●● → OK   
            
        """ 均時差の計算(A.3) """
            # eed:均時差[hour]
        eed = SCFModule.calc_eed(NDay)
        # print(Hour00,eed)  # ●● ←デバッグ用
            # ●● → \確認(全体動作).xlsx「eed」シートで確認
            # ●● → OK           
        
        """ 「月」の計算(D.2) """
        Month = c.get_month(Hour00)
            # ●● 省略
        
        """ +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ (1/NDT)時間のループ ++++ """ 
            # MM:1時間の内の(1/NDT)間隔の順番, 正時がMM=0, MM=0～NDT-1
        for MM in range(NDT):
            """ 時刻TTの計算(A.1) """
                # TT:時間分割MM毎の時刻[hour]
            TT = SCFModule.calc_TT(NHour, NDT, MM)

            """ 時角の計算(A.4) """
                # Tdt:時角[deg]
            Tdt = SCFModule.calc_Tdt(Longitude, eed, TT)
            # print(Hour00,MM,Tdt)  # ●● ←デバッグ用
                # ●● → \確認(全体動作).xlsx「Tdt」シートで確認
                # ●● → OK 
            
            """ 太陽高度とその正弦,余弦の計算(A.5,A.6) """
                # hsdt:太陽高度hsdt[deg], sinh:hsdtの正弦, cosh:hsdtの余弦
            sinh = SCFModule.calc_sinh(Latitude, deltad, Tdt)
            cosh = SCFModule.calc_cosh(sinh)
            hsdt = SCFModule.calc_hsdt(cosh, sinh)
            # print(Hour00,MM,sinh, cosh, hsdt)  # ●● ←デバッグ用
                # ●● → \確認(全体動作).xlsx「hsdt」シートで確認
                # ●● → OK             
            
            """ 太陽方位角の計算(A.7) """
                # Azsdt:太陽方位角[deg]
            Azsdt = SCFModule.calc_Azsdt(Latitude, deltad, Tdt, sinh, cosh)
            
            """ 窓面の法線ベクトルと太陽位置とのなす水平面上の角度の計算(A.9) """
                # Azwjdt:窓面の法線ベクトルと太陽位置とのなす水平面上の角度[deg]
            Azwjdt = SCFModule.calc_Azwjdt(Azwj, Azsdt)
            # print(Hour00,MM,Azsdt, Azwjdt)  # ●● ←デバッグ用
                # ●● → \確認(全体動作).xlsx「Azsdt」シートで確認
                # ●● → OK              
            
            """ (1/NDT)分割MM番目の法線面直達日射量, 水平面天空日射量(D.4) """
                # Sddhm:(1/NDT)分割MM番目における法線面直達日射量[kcal/m2]
                # Ssdhm:(1/NDT)分割MM番目における水平面天空日射量[kcal/m2]
            Sddhm = SCFModule.calc_Sdhm(MM, NDT, sinh, SRHour[Hour00][1], SRHour[Hour00+1][1]
                    , Nh[Hour00], Nh[Hour00+1])
            Ssdhm = SCFModule.calc_Sdhm(MM, NDT, sinh, SRHour[Hour00][2], SRHour[Hour00+1][2]
                    , Nh[Hour00], Nh[Hour00+1]) 
            # print(Hour00,MM,Sddhm, Ssdhm)  # ●● ←デバッグ用
                # ●● → \確認(全体動作).xlsx「SddhmSsdhm」シートで確認
                # ●● → OK                
            
            """ 直達日射の入射角(D.6) """
                # costheta:直達入射の窓面への入射角の余弦  
            costheta = SCFModule.calc_costheta(Azwjdt, cosh)
   
            """ 直達日射に対する窓ガラスの入射角特性(D.6) """
                # etajdt:日付NDay,時刻TTにおける入射角特性
            etajdt = SCFModule.calc_etajdt(costheta, etakk)
            # print(Hour00,MM,costheta,etajdt)  # ●● ←デバッグ用
                # ●● → \確認(全体動作).xlsx「costheta,etajdt」シートで確認
                # ●● → OK   
            
            """ 直達日射が窓に射す部分の面積の計算(B.7) """
                # Ax:直達日射が窓に射す部分の面積[m2]
            Ax = SCFModule.calc_Ax(WSSize, Azwjdt, hsdt)
            # print(Hour00,MM,Ax if Sddhm>0 else 0)  # ●● ←デバッグ用
            # ●● → \確認(全体動作).xlsx「Ax」シートで確認
                # ●● → Sddhm > 0 の条件を課す(実際の問題にならない)ことで、 
                # ●●    \日除け効果係数算定シート(全対応)-9.xlsm 「算定」シート DN列 と一致 → OK
            
            """ 日よけ効果係数算定式の時刻TTにおける分子分母(E.2) """
                # SCF00[0]:(2)式の分子への加算分(窓面積をかけた値として)     
                # SCF00[1]:(2)式の分母への加算分(窓面積をかけた値として)
            [SCF01[0][Hour00][MM],SCF01[1][Hour00][MM]] \
                = SCFModule.calc_SCF00(Sddhm, Ssdhm, etajdt, etaisr, costheta, sinh, Awj, Ax, gammayp, gammaym)
            # print(Hour00,MM,SCF01[0][Hour00][MM],SCF01[1][Hour00][MM])  # ●● ←デバッグ用        
                # ●● → \確認(全体動作).xlsx「SCF01」シートで確認
                # ●● → OK  
            
    SCF = SCFModule.Output_ShadingCorrectionFactor(SRHour, NDT, SCF01)
    
#     return SCF   # ●● ←デバッグ用にコメントアウト ●●

    # SCF[h][i][j]
    # h = 0：分子, h = 1：分母, h = 2：効果係数
    # i = -2：冷房期積算, i = -1：暖房期積算, i = 0：非空調期積算,
    #         i = 1～12：各月で積算
    # j = -1：日積算, j = 0～24：各時刻で積算             

print('冷房期 = {}, 暖房期 = {}, 非空調期 = {}'.format( SCF[2][-2][-1], SCF[2][-1][-1], SCF[2][0][-1] ))
for i in range(1,13):
    for j in range(25):
        print(i,j,SCF[0][i][j],SCF[1][i][j],SCF[2][i][j])
            # ●● → \確認(全体動作).xlsx「SCF01」シートの52568行以下で確認
            # ●● → OK

#### X.2.3 \日除け効果係数算定シート(全対応)-9.xlsm の「算定」シート セルEC8～FD29 全96ケースで確認(Test兼用)

In [ ]:
""" 日よけ効果係数計算プログラム本体 Test """
# \確認.xlsx "本体テスト1"シート → \AllTest01.csv を作成 → \TestConfig01 下に置いて読み込み → 計算

import Shading_Correction_Factor_Main as SCFMain
import pandas as pd
import sys
csv_input = pd.read_csv(filepath_or_buffer="./TestConfig01/AllTest01.csv", encoding="ms932", sep=",")
if csv_input.columns[0]!="AllTest01_case":
    sys.exit("ファイル内に貼り付けたテスト条件が違います")
for i in range(len(csv_input)):
    [case, Path00, FileName00, FileName01, ClimateZone, NDT, etaID, Azimuth] \
        = csv_input.values[i][0:8]
    WSSize = csv_input.values[i][8:-2]
    [SCFhA, SCFcA] = csv_input.values[i][-2:]
    SCF = SCFMain.Calc_ShadingCorrectionFactor(Path00, FileName00, FileName01
                                               , ClimateZone, NDT, etaID, Azimuth, WSSize )
        # SCF[h][i][j]
        # h = 0：分子, h = 1：分母, h = 2：効果係数
        # i = -2：冷房期積算, i = -1：暖房期積算, i = 0：非空調期積算,
        #         i = 1～12：各月で積算
        # j = -1：日積算, j = 0～24：各時刻で積算             
    [SCFh, SCFc] = [SCF[2][-1][-1], SCF[2][-2][-1]]
    print('case = {}: SCFh = {}, 期待値 = {}, 残差 = {}, SCFc = {}, 期待値 = {}, 残差 = {}'
          .format( case, SCFh, SCFhA, SCFh - SCFhA, SCFc, SCFcA, SCFc - SCFcA ))